In [1]:
import pandas as pd
import itertools as it
import gurobipy as gp
from gurobipy import GRB

In [2]:
data1 = [2000, 1000, 500, 0,
         1500, 1400, 2000, 2500,
         1000, 1000, 1500, 2000]
data2 = [0.25, 0.20, 0.10,
         0.10, 0.05, 0.05]

years = list(range(1, 4))
categ = ['Unskilled', 'Semi', 'Skilled']
m_req = dict(zip(it.product(categ, [0]+years), data1))
waste = dict(zip(it.product(['less', 'more'], categ), data2))

recruit_cap = dict(zip(categ, [500, 800, 500]))
retrain_unsk_semi_cost = 400
retrain_unsk_semi_cap  = 200
retrain_semi_skil_cost = 500
retrain_semi_skil_cap  = 0.25

downgrad_waste = 0.5

redund_unsk_pay = 200
redund_semi_pay = 500

overman_cost = dict(zip(categ, [1500, 2000, 3000]))
overman_cap  = 150

shorttime_cap  = 50
shorttime_cost = dict(zip(categ, [500, 400, 400]))

In [41]:
model = gp.Model('Manpower Planning')

# add vars

    # total number of workers, each year
t = model.addVars(categ, [0]+years,
                  name='tot')
    # people recruited, each year
u = model.addVars(categ, years,
                  name='rec')
    # workers retrained, each year
v = model.addVars(['u_semi', 'semi_s'], years,
                  name='ret')
    # downgraded workers, each year
r = model.addVars(['s_semi', 's_u', 'semi_u'], years,
                  name='down')
    # redundant workers, each year
w = model.addVars(categ, years,
                  name='red')
    # short-time workers, each year
x = model.addVars(categ, years,
                  name='short')
    # overmanning, excess workers, each year
y = model.addVars(categ, years,
                  name='over')

# set initial conditions and upper bounds
for i in years:
    for c in categ:
        u[c,i].ub = recruit_cap[c]
        x[c,i].ub = shorttime_cap
        t[c,0] = m_req[c,0]
        
    v['u_semi', i].ub = retrain_unsk_semi_cap
        
    
# objective function

    # minimize redundancy
model.setObjective((gp.quicksum(w[c,i] for c in categ for i in years)), GRB.MINIMIZE)
    
    # minimize cost
#model.setObjective((gp.quicksum(400*v['u_semi',i] + 500*v['semi_s',i] + 200*w['Skilled',i] + 500*w['Semi',i] + 500*w['Unskilled',i] + 500*x['Skilled',i] + 400*x['Semi',i] + 400*x['Unskilled',i] + 1500*y['Skilled',i] + 2000*y['Semi',i] + 3000*y['Unskilled',i] for i in years)), GRB.MINIMIZE)

# add constraints

    # continuity (connect number of workers from previous year to current year)
        # skilled
c = 'Skilled'
model.addConstrs((0.95*t[c,i-1] + 0.9*u[c,i]
                + 0.95*v['semi_s',i] 
                - r['s_semi',i] - r['s_u',i] 
                - w[c,i] == t[c,i] for i in years),
                 name='cont_skil')
        # semi-skilled
c = 'Semi'
model.addConstrs((0.95*t[c,i-1] + 0.8*u[c,i]
                + 0.95*v['u_semi',i] - v['semi_s',i]
                + 0.5*r['s_semi',i] - r['semi_u',i] 
                - w[c,i] == t[c,i] for i in years),
                 name='cont_semi')
        # unskilled
c = 'Unskilled'
model.addConstrs((0.9*t[c,i-1] + 0.75*u[c,i]
                - v['u_semi',i] 
                + 0.5*r['s_u',i] + 0.5*r['semi_u',i]
                - w[c,i] == t[c,i] for i in years),
                 name='cont_unsk')
    
    # retraining semi-skilled to skilled
model.addConstrs((v['semi_s',i] - 0.25*t['Skilled',i] <= 0 for i in years),
                 name='ret_semi')
    
    # overmanning limit
model.addConstrs((gp.quicksum(y[c,i] for c in categ) <= overman_cap for i in years),
                 name='overman')
    
    # workers requirements
model.addConstrs((t[c,i] - y[c,i] - 0.5*x[c,i] == m_req[c,i] 
                  for c in categ for i in years ),
                 name='requir')

model.update()
model.write('Manpower Planning.lp')
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 24 rows, 63 columns and 105 nonzeros
Model fingerprint: 0xd73bbcf3
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+01, 8e+02]
  RHS range        [2e+02, 3e+03]
Presolve removed 12 rows and 35 columns
Presolve time: 0.01s
Presolved: 12 rows, 28 columns, 56 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4000000e+02   6.484375e+01   0.000000e+00      0s
       8    8.4179688e+02   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.03 seconds
Optimal objective  8.417968750e+02


In [42]:
print('Objective value of {:.2f}'.format(model.ObjVal))

Objective value of 841.80


In [43]:
print('Available workforce')
report = pd.DataFrame([], columns=categ, index=years)
for c,i in t.keys():
    if i>0:
        report[c][i] = t[c,i].x
report

Available workforce


,Unskilled,Semi,Skilled
1,1157.03125,1442.96875,1025.0
2,675.0,2000.0,1500.0
3,175.0,2500.0,2000.0


In [44]:
print('Treinament plan')
report = pd.DataFrame([], columns=['u_semi', 'semi_s'], index=years)
for c,i in v.keys():
    if i>0:
        report[c][i] = v[c,i].x
report

Treinament plan


,u_semi,semi_s
1,200.0,256.25
2,200.0,80.263158
3,200.0,131.578947


In [45]:
print('Recruitment plan')
report = pd.DataFrame([], columns=categ, index=years)
for c,i in t.keys():
    if i>0:
        report[c][i] = u[c,i].x
report

Recruitment plan


,Unskilled,Semi,Skilled
1,0.0,0.0,0.0
2,0.0,649.303557,500.0
3,0.0,676.973684,500.0
